In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from pprint import pprint

DATA_PATH = 'D:\download'
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)



#transactions_exp = pd.read_csv(
#    os.path.join(DATA_PATH, 'transactions_exp.csv'),
    
#)


ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)
ratings.head()

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07


In [26]:

transactions_exp = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_exp2.csv'),
    
)

In [27]:
transactions_exp=transactions_exp.replace([np.inf], np.nan)
transactions_exp.watched_time=transactions_exp.watched_time.fillna(1.0)

In [28]:
transactions_exp['feature_5_65']=transactions_exp['feature_5_65'].astype(np.int16)
transactions_exp['feature_5_00']=transactions_exp['feature_5_00'].astype(np.int16)
transactions_exp['feature_5_68']=transactions_exp['feature_5_68'].astype(np.int16)
transactions_exp['feature_5_59']=transactions_exp['feature_5_59'].astype(np.int16)
transactions_exp['feature_5_-1']=transactions_exp['feature_5_-1'].astype(np.int16)
transactions_exp['smallf3']= transactions_exp['smallf3'].astype(np.int16)
transactions_exp['middlef3']=transactions_exp['middlef3'].astype(np.int16)
transactions_exp['bigf3']=transactions_exp['bigf3'].astype(np.int16)

transactions_exp['type_m']=transactions_exp['type_m'].astype(np.int16)

transactions_exp['type_s']=transactions_exp['type_s'].astype(np.int16)

transactions_exp['type_mm']= transactions_exp['type_mm'].astype(np.int16)

transactions_exp['feature_3']=transactions_exp['feature_3'].astype(np.int16)

transactions_exp['duration']=transactions_exp['duration'].astype(np.int16)

transactions_exp['element_uid']=transactions_exp['element_uid'].astype(np.int32)
transactions_exp['user_uid']=transactions_exp['user_uid'].astype(np.int32)
transactions_exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9643012 entries, 0 to 9643011
Data columns (total 33 columns):
Unnamed: 0        int64
Unnamed: 0.1      int64
Unnamed: 0.1.1    int64
element_uid       int32
user_uid          int32
watched_time      float64
rating            float64
duration          int16
feature_1         float64
feature_2         float64
feature_3         int16
feature_4         float64
quantity          int64
rating_mean       float64
CM_S              int64
CM_P              int64
CM_R              int64
feature_5_65      int16
feature_5_00      int16
feature_5_68      int16
feature_5_59      int16
feature_5_-1      int16
feature_5_44      int64
smallf3           int16
middlef3          int16
bigf3             int16
type_m            int16
type_s            int16
type_mm           int16
longfilm          int64
shortfilm         int64
worst             int64
watched           int64
dtypes: float64(6), int16(13), int32(2), int64(12)
memory usage: 1.6 GB


In [29]:
transactions_exp=transactions_exp.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'feature_1', 'worst'],1)
print(np.isinf(transactions_exp).sum())
transactions_exp.info()

element_uid     0
user_uid        0
watched_time    0
rating          0
duration        0
feature_2       0
feature_3       0
feature_4       0
quantity        0
rating_mean     0
CM_S            0
CM_P            0
CM_R            0
feature_5_65    0
feature_5_00    0
feature_5_68    0
feature_5_59    0
feature_5_-1    0
feature_5_44    0
smallf3         0
middlef3        0
bigf3           0
type_m          0
type_s          0
type_mm         0
longfilm        0
shortfilm       0
watched         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9643012 entries, 0 to 9643011
Data columns (total 28 columns):
element_uid     int32
user_uid        int32
watched_time    float64
rating          float64
duration        int16
feature_2       float64
feature_3       int16
feature_4       float64
quantity        int64
rating_mean     float64
CM_S            int64
CM_P            int64
CM_R            int64
feature_5_65    int16
feature_5_00    int16
feature_5_68    int16
feature_

In [30]:


X_train = transactions_exp[~transactions_exp.rating.isnull()]
X_train=X_train.drop(['rating'],1)
X_test = transactions_exp[transactions_exp.rating.isnull()]
X_test=X_test.drop(['rating'],1)
y_train = transactions_exp[~transactions_exp.rating.isnull()].rating.astype(np.float)
y_test = transactions_exp[transactions_exp.rating.isnull()].rating
X_test.rating_mean=X_test.rating_mean.fillna(6.0) #попробовать на всех

In [31]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))


X_test=X_test.drop([
    'rating_mean',
'watched_time',    
'duration'  ,      
'feature_2'  ,   
'feature_3'  ,   
'feature_4'  ,   
'quantity'  ,    
'CM_S'  ,        
'CM_P',  
'CM_R'  ,        
'feature_5_65' , 
'feature_5_00'  ,
'feature_5_68'  ,
'feature_5_59'  ,
'feature_5_-1'  ,
'feature_5_44'  ,
'smallf3' ,      
'middlef3'  ,    
'bigf3',         
'type_m',     
'type_s'  ,      
'type_mm' ], 1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)
X_test=X_test.drop(['watched'], 1)
X_test=X_test.drop(['longfilm', 'shortfilm'], 1)

OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
OLD2 = OLD.append(ratings)




In [32]:
print(3)

3


In [33]:
del transactions_exp

transactions_exp = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions_exp.csv'),
    
)
transactions_exp1=transactions_exp

transactions_exp1['worst']=transactions_exp1.watched_time.map(lambda x:1 if x < 0.2 and x >0.005 else 0)
transactions_exp1['watched']=transactions_exp1.watched_time.map(lambda x: 0 if x < 0.51 else 1)

Test=transactions_exp1[transactions_exp1['worst'] == 1]   
                                                                   


submit2=Test[Test.rating.isnull() == True]
submit2.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1722310 entries, 24 to 9642999
Data columns (total 32 columns):
Unnamed: 0      int64
Unnamed: 0.1    int64
element_uid     int64
user_uid        int64
watched_time    float64
rating          float64
duration        int64
feature_1       float64
feature_2       float64
feature_3       int64
feature_4       float64
quantity        int64
rating_mean     float64
CM_S            int64
CM_P            int64
CM_R            int64
feature_5_65    int64
feature_5_00    int64
feature_5_68    int64
feature_5_59    int64
feature_5_-1    int64
feature_5_44    int64
smallf3         int64
middlef3        int64
bigf3           int64
type_m          int64
type_s          int64
type_mm         int64
longfilm        int64
shortfilm       int64
worst           int64
watched         int64
dtypes: float64(6), int64(26)
memory usage: 433.6 MB


In [34]:
submit2['feature_5_65']=submit2['feature_5_65'].astype(np.int16)
submit2['feature_5_00']=submit2['feature_5_00'].astype(np.int16)
submit2['feature_5_68']=submit2['feature_5_68'].astype(np.int16)
submit2['feature_5_59']=submit2['feature_5_59'].astype(np.int16)
submit2['feature_5_-1']=submit2['feature_5_-1'].astype(np.int16)
submit2['smallf3']= submit2['smallf3'].astype(np.int16)
submit2['middlef3']=submit2['middlef3'].astype(np.int16)
submit2['bigf3']=submit2['bigf3'].astype(np.int16)

submit2['type_m']=submit2['type_m'].astype(np.int16)

submit2['type_s']=submit2['type_s'].astype(np.int16)

submit2['type_mm']= submit2['type_mm'].astype(np.int16)

submit2['feature_3']=submit2['feature_3'].astype(np.int16)

submit2['duration']=submit2['duration'].astype(np.int16)

submit2['element_uid']=submit2['element_uid'].astype(np.int32)
submit2['user_uid']=submit2['user_uid'].astype(np.int32)


submit2=submit2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'feature_1'],1)
submit2=submit2.drop(['worst', 'rating'],1)
submit2.rating_mean=submit2.rating_mean.fillna(6.0)

submit22 = pd.DataFrame(model2.predict(submit2))


submit2=submit2.drop([
    'rating_mean',
'watched_time',    
'duration'  ,      
'feature_2'  ,   
'feature_3'  ,   
'feature_4'  ,   
'quantity'  ,    
'CM_S'  ,        
'CM_P',  
'CM_R'  ,        
'feature_5_65' , 
'feature_5_00'  ,
'feature_5_68'  ,
'feature_5_59'  ,
'feature_5_-1'  ,
'feature_5_44'  ,
'smallf3' ,      
'middlef3'  ,    
'bigf3',         
'type_m',     
'type_s'  ,      
'type_mm' ], 1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
submit2=submit2.reset_index()
submit2=submit2.drop(['index'], 1)
submit2=submit2.drop(['watched'], 1)
submit2=submit2.drop(['longfilm', 'shortfilm'], 1)

OLD333=pd.concat([submit2, submit22], 1)
#OLD.rating=OLD.rating.round()
OLD333['rating']=OLD333[0]
OLD333=OLD333.drop([0],1)
#ratings=ratings.drop(['ts'],1)
OLD2 = OLD2.append(OLD333)


OLD2.to_csv(os.path.join(DATA_PATH, 'OLD2(doobuch).csv'))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [19]:
submit2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1753282 entries, 24 to 9642999
Data columns (total 26 columns):
element_uid     int32
user_uid        int32
watched_time    float64
duration        int16
feature_2       float64
feature_3       int16
feature_4       float64
quantity        int64
rating_mean     float64
CM_S            int64
CM_P            int64
CM_R            int64
feature_5_65    int16
feature_5_00    int16
feature_5_68    int16
feature_5_59    int16
feature_5_-1    int16
feature_5_44    int64
smallf3         int16
middlef3        int16
bigf3           int16
type_m          int16
type_s          int16
type_mm         int16
longfilm        int64
shortfilm       int64
dtypes: float64(4), int16(13), int32(2), int64(7)
memory usage: 217.4 MB


In [8]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor


kf = KFold(n_splits=5, shuffle=True, random_state=241) 


model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241, n_jobs=-1)
score=cross_val_score(model2, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
print("Score is", score.mean())
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(estimator.predict(X_test))
submit1.to_csv("sample_submissionrf02.03.csv")


estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )
score1=cross_val_score(estimator, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
print("Score is", score1.mean())
estimator.fit(X_train, y_train)
submit = pd.DataFrame(estimator.predict(X_test))
submit.to_csv("sample_submissionxgb02.03.csv")



D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Score is -0.625218053017624


NameError: name 'estimator' is not defined

In [9]:
estimator=xgb.XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=700, n_jobs=-1  )
score1=cross_val_score(estimator, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
print("Score is", score1.mean())

Score is -0.614842092771812


In [10]:
print('start')
model2.fit(X_train, y_train)
submit1 = pd.DataFrame(model2.predict(X_test))
submit1.to_csv("sample_submissionrf02.03.csv")
print('one')
estimator.fit(X_train, y_train)
submit = pd.DataFrame(estimator.predict(X_test))
submit.to_csv("sample_submissionxgb02.03.csv")
print('two')

start
one
two


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241)
score=cross_val_score(model2, X_train, y_train, cv=kf, n_jobs=-1, scoring='mean_squared_error')
#model2=model2.fit(X_train, y_train)
#submit = pd.DataFrame(model2.predict(X_test)) #500 и 8 в итоге заиграли

In [28]:
estimator.fit(X_train, y_train)
submit = pd.DataFrame(estimator.predict(X_test))
submit.to_csv("sample_submissionxgb.csv")


In [11]:
Score is -3.327374027489791
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV

kf = KFold(n_splits=5, shuffle=True, random_state=241) 


grid = {'n_estimators': [521,522,523]}
model = RandomForestRegressor( max_depth=4, random_state=241)
gs = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1)
gs.fit(X_train, y_train)

Cb=gs.best_params_['n_estimators']
Sb=gs.best_score_ 
print (Cb, Sb, 'это поиск по деревьям')



522 -3.3525222034319064 это поиск по деревьям


In [12]:

grid1 = {'max_depth': [6,7,8,9]}
model1 = RandomForestRegressor( n_estimators=Cb, random_state=241)
gs1 = GridSearchCV(model1, grid1, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1)
gs1.fit(X_train, y_train)
Cb1=gs1.best_params_['max_depth']
Sb1=gs1.best_score_
print (Cb1, Sb1, 'это поиск по глубине')


8 -3.336118804707078 это поиск по глубине


In [39]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor( n_estimators=522, max_depth=8, random_state=241)
model2=model2.fit(X_train, y_train)
submit = pd.DataFrame(model2.predict(X_test)) #500 и 8 в итоге заиграли

In [35]:
importances = model2.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names=X_train.columns
print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5}' ({:.4f})".format(f + 1, feature_names[idx], importances[idx]))


Feature importances:
 1. feature 'watched_time' (0.9713)
 2. feature 'rating_mean' (0.0256)
 3. feature 'CM_S ' (0.0015)
 4. feature 'quantity' (0.0005)
 5. feature 'user_uid' (0.0002)
 6. feature 'duration' (0.0002)
 7. feature 'CM_P ' (0.0002)
 8. feature 'CM_R ' (0.0001)
 9. feature 'feature_2' (0.0001)
10. feature 'feature_3' (0.0001)
11. feature 'element_uid' (0.0001)
12. feature 'feature_4' (0.0001)
13. feature 'type_mm' (0.0000)
14. feature 'type_m' (0.0000)
15. feature 'feature_5_44' (0.0000)
16. feature 'feature_5_65' (0.0000)
17. feature 'longfilm' (0.0000)
18. feature 'type_s' (0.0000)
19. feature 'feature_5_00' (0.0000)
20. feature 'feature_5_68' (0.0000)
21. feature 'feature_5_59' (0.0000)
22. feature 'middlef3' (0.0000)
23. feature 'feature_5_-1' (0.0000)
24. feature 'shortfilm' (0.0000)
25. feature 'bigf3' (0.0000)
26. feature 'smallf3' (0.0000)
27. feature 'watched' (0.0000)


In [13]:

from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor( n_estimators=500, max_depth=8, random_state=241)
model2=model2.fit(X_train, y_train)
submit = pd.DataFrame(model2.predict(X_test)) #500 и 8 в итоге заиграли

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV

kf = KFold(n_splits=5, shuffle=True, random_state=241) 
grid3 = {'n_neighbors': [28,29,30,31,32,33]}

neigh = KNeighborsRegressor()
gs3 = GridSearchCV(neigh, grid3, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1)
gs3.fit(X_train, y_train)
Cb3=gs3.best_params_['n_neighbors']
Sb3=gs3.best_score_
print (Cb3, Sb3, 'это поиск по соседям')

32 -3.8074985438316618 это поиск по соседям


In [12]:
X_test=X_test.drop([
    'rating_mean',
'watched_time',    
'duration'  ,      
'feature_2'  ,   
'feature_3'  ,   
'feature_4'  ,   
'quantity'  ,    
'CM_S'  ,        
'CM_P',  
'CM_R'  ,        
'feature_5_65' , 
'feature_5_00'  ,
'feature_5_68'  ,
'feature_5_59'  ,
'feature_5_-1'  ,
'feature_5_44'  ,
'smallf3' ,      
'middlef3'  ,    
'bigf3',         
'type_m',     
'type_s'  ,      
'type_mm' ], 1)
#submit[0]=submit[0].round()
#submit[0]=submit[0].astype(np.int16)
X_test=X_test.reset_index()
X_test=X_test.drop(['index'], 1)
X_test=X_test.drop(['watched'], 1)
X_test=X_test.drop(['longfilm', 'shortfilm'], 1)

OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
OLD2 = OLD.append(ratings)


transactions_exp=transactions_exp.drop([
    'rating_mean',
'watched_time',    
'duration'  ,      
'feature_2'  ,   
'feature_3'  ,   
'feature_4'  ,   
'quantity'  ,    
'CM_S'  ,        
'CM_P',  
'CM_R'  ,        
'feature_5_65' , 
'feature_5_00'  ,
'feature_5_68'  ,
'feature_5_59'  ,
'feature_5_-1'  ,
'feature_5_44'  ,
'smallf3' ,      
'middlef3'  ,    
'bigf3',         
'type_m',     
'type_s'  ,      
'type_mm',
'Unnamed: 0',
'Unnamed: 0.1',
'Unnamed: 0.1.1',
'feature_1',
'longfilm',
'shortfilm',
'watched',
'worst' ], 1)

OLD2=OLD2.append(transactions_exp)


OLD2.to_csv(os.path.join(DATA_PATH, 'OLD_tr_1.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6962767 entries, 0 to 6962766
Data columns (total 3 columns):
element_uid    int32
user_uid       int32
watched        int64
dtypes: int32(2), int64(1)
memory usage: 106.2 MB


In [14]:
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
OLD2 = OLD.append(ratings)

transactions_exp=transactions_exp.drop([
    'rating_mean',
'watched_time',    
'duration'  ,      
'feature_2'  ,   
'feature_3'  ,   
'feature_4'  ,   
'quantity'  ,    
'CM_S'  ,        
'CM_P',  
'CM_R'  ,        
'feature_5_65' , 
'feature_5_00'  ,
'feature_5_68'  ,
'feature_5_59'  ,
'feature_5_-1'  ,
'feature_5_44'  ,
'smallf3' ,      
'middlef3'  ,    
'bigf3',         
'type_m',     
'type_s'  ,      
'type_mm',
'Unnamed: 0',
'Unnamed: 0.1',
'Unnamed: 0.1.1',
'feature_1',
'longfilm',
'shortfilm',
'watched',
'worst' ], 1)

OLD2=OLD2.append(transactions_exp)


OLD2.to_csv(os.path.join(DATA_PATH, 'OLD_tr_1.csv'))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7401557 entries, 0 to 438789
Data columns (total 4 columns):
element_uid    int32
rating         float32
ts             float64
user_uid       int64
dtypes: float32(1), float64(1), int32(1), int64(1)
memory usage: 225.9 MB


In [15]:
OLD=pd.concat([X_test, submit1], 1)
#OLD.rating=OLD.rating.round()
OLD['rating']=OLD[0]
OLD=OLD.drop([0],1)
#ratings=ratings.drop(['ts'],1)
OLD2 = OLD.append(ratings)
OLD2.to_csv(os.path.join(DATA_PATH, 'OLD(02.03RF).csv'))
OLD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7401557 entries, 0 to 438789
Data columns (total 4 columns):
element_uid    int32
rating         float64
ts             float64
user_uid       int64
dtypes: float64(2), int32(1), int64(1)
memory usage: 254.1 MB


In [43]:
OLD2.to_csv(os.path.join(DATA_PATH, 'OLD2(1).csv'))

In [44]:
OLD2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724459 entries, 0 to 438789
Data columns (total 4 columns):
element_uid    int64
rating         float64
ts             float64
user_uid       int64
dtypes: float64(2), int64(2)
memory usage: 371.0 MB


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import mglearn
%matplotlib inline
from IPython.display import display
(transactions_exp.isnull().sum()/transactions_exp.shape[0]).plot(kind='bar');